In [1]:
import pandas as pd
import numpy as np
import re
from datetime import timedelta
from connection_keys import connect_bq_client
import statsmodels.api as sm
import os
import warnings
import itertools
warnings.filterwarnings("ignore")

In [2]:
client = connect_bq_client()

sql = """
    SELECT *
    FROM `db_market_product.gpu`
    ORDER BY dt_carga asc
"""

df = client.query(sql).to_dataframe()

In [3]:
df['data'] = df.dt_carga.dt.date
df.head()

,ds_marca,ds_nome_produto,vl_preco_atual,vl_preco_com_desconto,vl_porcentagem_do_desconto,qt_quantidade_estoque,oferta,ds_disponibilidade,loja,dt_carga,data
0,Nvidia,Nvlink 3 Slot Rtx Geforce,1322.0,1322.0,0.0,0,None,False,Kabum,2022-09-29 12:26:15.112196+00:00,2022-09-29
1,Gigabyte,"Gigabyte GTX 1050 Ti, 4GB, DDR5",1999.0,1999.0,0.0,0,None,False,Kabum,2022-09-29 12:26:15.112196+00:00,2022-09-29
2,Nvidia,Vga Pci-e 1gb Galax Geforce Gt710 - 1gb,339.0,339.0,0.0,0,None,False,Kabum,2022-09-29 12:26:15.112196+00:00,2022-09-29
3,Colorful,"Placa de Video Colorful GT 710, 1GB DDR3",349.0,349.0,0.0,93,None,True,Kabum,2022-09-29 12:26:15.112196+00:00,2022-09-29
4,Asus,Placa de Vídeo Asus GeForce RTX 3080 10GB,19999.0,19999.0,0.0,0,None,False,Kabum,2022-09-29 12:26:15.112196+00:00,2022-09-29


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340247 entries, 0 to 340246
Data columns (total 11 columns):
 #   Column                      Non-Null Count   Dtype              
---  ------                      --------------   -----              
 0   ds_marca                    340247 non-null  object             
 1   ds_nome_produto             340247 non-null  object             
 2   vl_preco_atual              340247 non-null  float64            
 3   vl_preco_com_desconto       340247 non-null  float64            
 4   vl_porcentagem_do_desconto  340247 non-null  float64            
 5   qt_quantidade_estoque       126440 non-null  Int64              
 6   oferta                      340247 non-null  object             
 7   ds_disponibilidade          340247 non-null  object             
 8   loja                        340247 non-null  object             
 9   dt_carga                    340247 non-null  datetime64[ns, UTC]
 10  data                        340247 non-null 

In [33]:
df.groupby(['dt_carga'])['loja'].count()

# df['data'].value_counts().sort_index().reset_index()
# df['data'] = df.dt_carga.dt.date

dt_carga
2022-09-29 12:26:15.112196+00:00    2663
2022-09-29 22:01:30.406181+00:00    2663
2022-09-30 10:01:23.630323+00:00    2663
2022-09-30 22:01:22.660381+00:00    2661
2022-10-01 10:02:27.606722+00:00    2662
                                    ... 
2022-11-26 10:01:22.800606+00:00    2777
2022-11-26 22:01:23.720246+00:00    2778
2022-11-27 10:01:27.473085+00:00    2697
2022-11-27 22:01:28.816066+00:00    2777
2022-11-28 10:01:30.097580+00:00    2687
Name: loja, Length: 129, dtype: int64

In [70]:
df_datas_loja = df.groupby(['data', 'loja'])['dt_carga'].nunique()
df_datas_loja

data        loja  
2022-09-29  Kabum     2
            Pichau    2
2022-09-30  Kabum     2
            Pichau    2
2022-10-01  Kabum     2
                     ..
2022-11-26  Pichau    2
2022-11-27  Kabum     2
            Pichau    2
2022-11-28  Kabum     1
            Pichau    1
Name: dt_carga, Length: 122, dtype: int64

In [71]:
df_datas_loja = df_datas_loja.to_frame(name='contagem')
df_datas_loja_unstack = df_datas_loja.unstack('loja')
# df_datas_loja_unstack.columns
# df_datas_loja_unstack_teste = df_datas_loja_unstack.columns.set_levels(['data','kabum','pichau'],level=1)
d = dict(zip(df.columns.levels[1], ))
# df_datas_loja_unstack_teste

AttributeError: 'Index' object has no attribute 'levels'

In [13]:
df.loc[df['data'].astype(str) == '2022-10-01']['dt_carga'].unique()
# df

<DatetimeArray>
['2022-10-01 10:02:27.606722+00:00', '2022-10-01 22:05:17.603092+00:00',
 '2022-10-01 22:07:59.247261+00:00']
Length: 3, dtype: datetime64[ns, UTC]

In [9]:
type(df['data'][0])

datetime.date